# Model Loading


In [1]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, load_in_8bit=True, device_map='auto')

/home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/samsong/anaconda3/envs/pytorchbook/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda115.so...


Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.39s/it]


In [2]:
input_text = "为什么再快的86也追不上坐着夏树的奔驰?"

ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')

outs = model.generate(ids, max_length=128)

response = tokenizer.batch_decode(outs)
print(response[0])

The dtype of attention mask (torch.int64) is not bool


为什么再快的86也追不上坐着夏树的奔驰? 这句话的表面意思是:即使86号车再快,也无法追上一辆坐着夏树(奔驰)的车。

这句话背后的含义是:速度与时间的关系。

86号车和夏树(奔驰)的车在速度上的差异,可能是因为86号车的速度比夏树(奔驰)的车慢,或者是因为夏树(奔驰)的车比86号车更快。此外,时间也是一个重要的因素。86号车和夏树(奔驰)的车在时间上的差异,可能是因为夏树(奔驰)的车比


# Add Lora

### Generate results without fine tuning

In [3]:
from peft import LoraConfig, TaskType, prepare_model_for_kbit_training, get_peft_model

model.gradient_checkpointing_enable()
model.is_parallelizable = True
model.model_parallel = True
model.config.use_cache = (
    False  # silence the warnings. Please re-enable for inference!
)

prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)
print(type(model))

<class 'peft.peft_model.PeftModelForCausalLM'>


In [4]:
datas = [
{
    "context": "大师，我最近事业不顺利，感觉很迷茫，该怎么办？",
    "target": "孩子，生活中的起伏和挫折是常态。放下烦恼，用平静的心态面对困境，努力奋斗，同时保持内心的善良与慈悲，你将找到属于你的光明之路。",
},
{
    "context": "师父，我与家人之间常常发生争吵，如何改善家庭关系？",
    "target": "阿弥陀佛，亲情是一份宝贵的财富，要从宽容和理解开始。培养平和的心态，倾听彼此的意见，以慈悲与智慧去化解冲突，共同创造和谐温暖的家庭氛围。",
},
{
    "context": "师傅，我经常感到焦虑和压力，怎样才能获得内心的宁静？",
    "target": "孩子，内心的宁静源于觉察和自我调适。修行冥想，深呼吸，寻找内心的栖息地。明白外在的变幻无常，放下过多的执着与欲望，你将发现内心的平静与安宁。",
},
{
    "context": "大师，我常常陷入烦恼和抱怨，怎样才能培养感恩之心？",
    "target": "施主，感恩是一种修养，需要从日常小事开始。每天意识到自己的幸运和获得，体察他人的善意和关爱。珍惜当下，感恩生活中的点滴，你将发现更多的快乐与满足。",
},
{
    "context": "师父，我对未来感到迷茫和恐惧，不知道自己的人生目标是什么，有何建议？",
    "target": "阿弥陀佛，人生的方向不一定要一目了然，它是通过不断尝试和探索而逐渐清晰的。保持对未知的勇敢与开放，寻找自己内心的声音。用心去体验，用智慧去选择，你将找到属于自己的道路。",
}]

In [5]:
for item in datas:
    text = f"问:{item['context']}\n答:"
    ids = tokenizer.encode(text, return_tensors='pt').to('cuda')
    outs = model.generate(input_ids=ids, max_length=128)
    print(tokenizer.batch_decode(outs)[0])

/home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


问:大师,我最近事业不顺利,感觉很迷茫,该怎么办?
答: 

首先,我很抱歉听到您的事业不顺利。但是,请不要放弃,因为事业迷茫是一种常见的情绪,很多人都会经历。以下是一些建议,希望对您有所帮助:

1. 找出问题所在:明确事业不顺利的原因。是因为缺乏灵感、缺乏创意、缺乏动力,还是其他原因?一旦确定了问题,就可以开始寻找解决方案。

2. 寻求帮助:如果感到迷茫,可以寻求专业人士的帮助,例如咨询师、心理医生或职业顾问。他们可以帮助理清思路,提供建议和指导。

3.
问:师父,我与家人之间常常发生争吵,如何改善家庭关系?
答: 家庭关系是人生中非常重要的一部分,而争吵是家庭中常见的问题。以下是一些可能有助于改善家庭关系的建议:

1. 沟通:沟通是解决冲突的关键。尝试与家人坦诚地交流,听取他们的观点,并尝试找到共同点。在交流过程中,尽量避免指责和攻击,而是尝试解决问题。

2. 尊重:尊重是建立和谐关系的基础。尊重家人的感受和观点,并尝试理解他们的立场。即使不同意他们的观点,也要尊重他们的意见,避免争吵。

3.
问:师傅,我经常感到焦虑和压力,怎样才能获得内心的宁静?
答: 焦虑和压力是常见的情绪体验,但可以通过一些方法来缓解和克服它们。以下是一些建议:

1. 深呼吸:深呼吸可以帮助你放松身体和心灵。尝试缓慢地吸气,然后慢慢地呼气,重复几次。

2. 冥想:冥想是一种放松和集中注意力的方法。找一个安静的地方,坐下来,闭上眼睛,专注于呼吸或一个特定的冥想练习。

3. 运动:运动可以帮助你释放紧张情绪和压力,并提高身体的代谢率。选择一种你喜欢的运动方式,
问:大师,我常常陷入烦恼和抱怨,怎样才能培养感恩之心?
答: 感恩之心是一种积极的情感,可以帮助我们更好地珍惜身边的人和事,并在生活中获得更多的快乐和满足感。以下是一些培养感恩之心的建议:

1. 意识到自己的感恩之心:要意识到感恩是一种积极的情感,而不是一种消极的情绪。当意识到自己正在感恩时,就可以开始思考身边的美好事物,并将它们纳入自己的行动计划中。

2. 感恩身边的人:感恩不仅仅是针对家人和亲人,还可以包括朋友、同事、邻居、社区和其他人。要时刻注意身边的美好事物,并表达感激之情。

3
问:师父,我对未来感到迷茫和恐惧,不知道自己的人生目标是什么,有何建议?
答: 对未来感到迷茫和恐惧是非常常见的感受,很多人

In [6]:
from tokenize_dataset_rows import preprocess
from datasets import Dataset

dataset = [preprocess(tokenizer, model.config, item, max_seq_length=256) for item in datas]

dataset = Dataset.from_list(dataset)


In [7]:
from finetune import ModifiedTrainer, data_collator
from transformers import TrainingArguments

training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 5,
    learning_rate = 1e-4,
    num_train_epochs=80,
    logging_steps=10,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/80 [00:00<?, ?it/s]/home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
 12%|█▎        | 10/80 [00:17<02:00,  1.72s/it]

{'loss': 4.0652, 'learning_rate': 9e-05, 'epoch': 10.0}


 25%|██▌       | 20/80 [00:34<01:43,  1.72s/it]

{'loss': 2.5896, 'learning_rate': 7.75e-05, 'epoch': 20.0}


 38%|███▊      | 30/80 [00:52<01:24,  1.69s/it]

...

 88%|████████▊ | 70/80 [02:01<00:17,  1.76s/it]

{'loss': 0.0311, 'learning_rate': 1.6250000000000002e-05, 'epoch': 70.0}


100%|██████████| 80/80 [02:19<00:00,  1.74s/it]

{'loss': 0.0237, 'learning_rate': 3.75e-06, 'epoch': 80.0}
{'train_runtime': 139.0364, 'train_samples_per_second': 2.877, 'train_steps_per_second': 0.575, 'train_loss': 1.0845098892226814, 'epoch': 80.0}


TrainOutput(global_step=80, training_loss=1.0845098892226814, metrics={'train_runtime': 139.0364, 'train_samples_per_second': 2.877, 'train_steps_per_second': 0.575, 'train_loss': 1.0845098892226814, 'epoch': 80.0})

### Generate results with LoRa

In [8]:
import torch

model.config.use_cache = (
    True
)

for item in datas:
    text = f"问:{item['context']}\n答:"
    ids = tokenizer.encode(text, return_tensors='pt')
    outs = model.generate(input_ids=ids, max_length=128)
    print(tokenizer.batch_decode(outs)[0])

/home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/samsong/anaconda3/envs/pytorchbook/lib/python3.8/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
`use_cache=True` is incompatible with gradient check

问:大师,我最近事业不顺利,感觉很迷茫,该怎么办?
答: 孩子,生活中的起伏和挫折是常态。放下烦恼,用平静的心态面对困境,努力奋斗,同时保持内心的善良与慈悲,你将找到属于你的光明之路。
问:师父,我与家人之间常常发生争吵,如何改善家庭关系?
答: 阿弥陀佛,亲情是一份宝贵的财富,要从宽容和理解开始。培养平和的心态,倾听彼此的意见,以慈悲与智慧去化解冲突,共同创造和谐温暖的家庭氛围。
问:师傅,我经常感到焦虑和压力,怎样才能获得内心的宁静?
答: 孩子,内心的宁静源于觉察和自我调适。修行冥想,深呼吸,寻找内心的栖息地。明白外在的变幻无常,放下过多的执着与欲望,你将发现内心的平静与安宁。
问:大师,我常常陷入烦恼和抱怨,怎样才能培养感恩之心?
答: 施主,感恩是一种修养,需要从日常小事开始。每天意识到自己的幸运和获得,体察他人的善意和关爱。珍惜当下,感恩生活中的点滴,你将发现更多的快乐与满足。
问:师父,我对未来感到迷茫和恐惧,不知道自己的人生目标是什么,有何建议?
答: 阿弥陀佛,人生的方向不一定要一目了然,它是通过不断尝试和探索而逐渐清晰的。保持对未知的勇敢与开放,寻找自己内心的声音。用心去体验,用智慧去选择,你将找到属于自己的道路。


# In-Context Learning

In [9]:
from peft.peft_model import PeftModelForCausalLM
type(model)

peft.peft_model.PeftModelForCausalLM

In [10]:
examples = [
    """原句子：一个优雅的粉白色百合在宁静的花园中绽放。柔软的花瓣轻轻带有粉色，而茎上浓绿的叶子形成了生机盎然的对比。
人：帮我将百合花改成玫瑰花
修改后：一个优雅的粉白色百合在宁静的花园中绽放。柔软的花瓣轻轻带有粉色，而茎上浓绿的叶子形成了生机盎然的对比。""",

"""原句子：一朵充满生机的仙人掌花盛开着，它的鲜艳粉红色花瓣和明亮的黄色芯与多刺的绿色仙人掌植物形成了对比。
人：请把花瓣涂成紫色
修改后：一朵充满生机的仙人掌花盛开着，它的鲜艳粉红色花瓣和明亮的黄色芯与多刺的绿色仙人掌植物形成了对比。""",

"""原句子：一个女孩头戴着一件白色宽松袖口的衬衫和一条长裙。她手持画笔，站在一个充满多彩花朵的花园中。背景是蓝天和蓬松的云朵。
人：请将多彩花朵替换为充满活力的花朵。
修改后：一个女孩头戴着一件白色宽松袖口的衬衫和一条长裙。她手持画笔，站在一个充满充满活力的花朵的花园中。背景是蓝天和蓬松的云朵。""",
]

In [21]:
input_text = f"""
原句子：两个演员在一场京剧表演中进行激动人心的剑术对决。他们的动作流畅而精确，以他们对这门艺术的掌握能够吸引观众。
人：请删除剑术对决的场景。

例如:
{' '.join(examples)}

"""

ids = tokenizer.encode(input_text, return_tensors='pt')
outs = model.base_model.generate(ids, max_length=512)
out_text = tokenizer.decode(outs[0])
print(out_text)



原句子:两个演员在一场京剧表演中进行激动人心的剑术对决。他们的动作流畅而精确,以他们对这门艺术的掌握能够吸引观众。
人:请删除剑术对决的场景。

例如:
原句子:一个优雅的粉白色百合在宁静的花园中绽放。柔软的花瓣轻轻带有粉色,而茎上浓绿的叶子形成了生机盎然的对比。
人:帮我将百合花改成玫瑰花
修改后:一个优雅的粉白色百合在宁静的花园中绽放。柔软的花瓣轻轻带有粉色,而茎上浓绿的叶子形成了生机盎然的对比。 原句子:一朵充满生机的仙人掌花盛开着,它的鲜艳粉红色花瓣和明亮的黄色芯与多刺的绿色仙人掌植物形成了对比。
人:请把花瓣涂成紫色
修改后:一朵充满生机的仙人掌花盛开着,它的鲜艳粉红色花瓣和明亮的黄色芯与多刺的绿色仙人掌植物形成了对比。 原句子:一个女孩头戴着一件白色宽松袖口的衬衫和一条长裙。她手持画笔,站在一个充满多彩花朵的花园中。背景是蓝天和蓬松的云朵。
人:请将多彩花朵替换为充满活力的花朵。
修改后:一个女孩头戴着一件白色宽松袖口的衬衫和一条长裙。她手持画笔,站在一个充满充满活力的花朵的花园中。背景是蓝天和蓬松的云朵。 原句子:两个演员在一场京剧表演中进行激动人心的剑术对决。他们的动作流畅而精确,以他们对这门艺术的掌握能够吸引观众。
人:删除剑术对决的场景。
修改后:两个演员在一场京剧表演中展现他们流畅而精确的动作,以他们对这门艺术的掌握能够吸引观众。
 两个演员在一场京剧表演中展现他们流畅而精确的动作,以他们对这门艺术的掌握能够吸引观众。
